In [22]:
import matplotlib.pyplot as plt 
import os
import pandas as pd
import numpy as np 
import scipy.cluster.hierarchy as shc
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
import warnings
import re 
pd.set_option("display.max_rows", 10)
pd.set_option("display.max_columns", 25)
pd.set_option("display.precision", 5)
import seaborn as sb
sb.set_style('whitegrid')
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')
current_dir=os.path.dirname(os.path.abspath("__file__"))
df0=pd.read_excel(os.path.join(current_dir, 'Дымный(добавил откатку).xlsx'), "Рабочий файл")


In [23]:
df=df0

In [24]:
df

,Сценарий,Блок,Передел,Объем,Тип техники,Марка техники+Инв,Марка техники,Инв,Производительность,Смена,Время выполнения,Дата 1,Дата 2
0,Наболее вероятный,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-06-01,NaT
1,Наболее вероятный,С1-1,разработка торфов,"4,244",Бульдозер,D-375/688,D-375,688,"2,500",1-2,2,2023-06-01,2023-06-02
2,Наболее вероятный,С1-1,погрузка торфов,"4,244",Экскаватор,PC-800 8/913,PC-800 8,913,"1,767",1,2,2023-06-02,2023-06-04
3,Наболее вероятный,С1-1,Транспортировка торфов,"1,415",Самосвал,БелАЗ/1082,БелАЗ,"1,082",589,1,2,2023-06-02,2023-06-04
4,Наболее вероятный,С1-1,Транспортировка торфов,"1,415",Самосвал,БелАЗ/1150,БелАЗ,"1,150",589,1,2,2023-06-02,2023-06-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,Наболее вероятный,С1-5 Бедовый,Транспортировка песков,"13,400",Самосвал,БелАЗ/659,БелАЗ,659,"1,200",1-2,11,2023-09-12,2023-09-23
307,Наболее вероятный,С1-5 Бедовый,Транспортировка песков,"13,400",Самосвал,БелАЗ/660,БелАЗ,660,"1,200",1-2,11,2023-09-12,2023-09-23
308,Наболее вероятный,С1-5 Бедовый,Промывка песков,"26,800",Промприбор,ГИТ-62 МЕ/668,ГИТ-62 МЕ,668,"2,400",1-2,11,2023-09-12,2023-09-23
309,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT


In [25]:

df=df.dropna()

In [26]:
# Устанавливаем тип данных для столбцов 'Дата 1' и 'Дата 2'
df['Дата 1'] = pd.to_datetime(df['Дата 1'], format='%Y.%m.%d.')
df['Дата 2'] = pd.to_datetime(df['Дата 2'], format='%Y.%m.%d.')

# Создаем пустой датафрейм для результатов
new_df = pd.DataFrame(columns=df.columns)

# Итерация по строкам датафрейма
for _, row in df.iterrows():
    dates = pd.date_range(start=row['Дата 1'], end=row['Дата 2'], freq='D')
    for date in dates:
        new_row = row.copy()
        new_row['Дата'] = date.strftime('%Y.%m.%d.')
        new_df = new_df.append(new_row, ignore_index=True)



C:\Users\poisk-12\AppData\Local\Temp\ipykernel_12144\3484977015.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Дата 1'] = pd.to_datetime(df['Дата 1'], format='%Y.%m.%d.')
C:\Users\poisk-12\AppData\Local\Temp\ipykernel_12144\3484977015.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Дата 2'] = pd.to_datetime(df['Дата 2'], format='%Y.%m.%d.')
C:\Users\poisk-12\AppData\Local\Temp\ipykernel_12144\3484977015.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pan

In [27]:
# Преобразование столбца 'Дата' в тип данных datetime
new_df['Дата'] = pd.to_datetime(new_df['Дата'])

In [28]:
# Извлечение названия месяца в новый столбец 'Месяц'
#new_df['Месяц'] = new_df['Дата'].dt.strftime('%B')


In [29]:
'''# Добавляем столбец "Уменьшающийся объем"
new_df['Уменьшающийся объем'] = new_df['Объем']
# Условная индексация для уменьшающегося объема
mask = (new_df['Передел'] == new_df['Передел'].shift(-1)) & \
       (new_df['Марка техники'] == new_df['Марка техники'].shift(-1)) & \
       (new_df['Блок'] == new_df['Блок'].shift(-1))
new_df.loc[mask, 'Уменьшающийся объем'] -= new_df['Производительность'].shift(-1)
'''

'# Добавляем столбец "Уменьшающийся объем"\nnew_df[\'Уменьшающийся объем\'] = new_df[\'Объем\']\n# Условная индексация для уменьшающегося объема\nmask = (new_df[\'Передел\'] == new_df[\'Передел\'].shift(-1)) &        (new_df[\'Марка техники\'] == new_df[\'Марка техники\'].shift(-1)) &        (new_df[\'Блок\'] == new_df[\'Блок\'].shift(-1))\nnew_df.loc[mask, \'Уменьшающийся объем\'] -= new_df[\'Производительность\'].shift(-1)\n'

In [30]:
for i, row in new_df.iterrows():
    
    if i>=1:     
        if  ((new_df.loc[i,"Передел"]==new_df.loc[i-1,"Передел"])& \
            (new_df.loc[i,"Марка техники"]==new_df.loc[i-1,"Марка техники"])& \
            (new_df.loc[i,"Блок"]==new_df.loc[i-1,"Блок"])& \
            (new_df.loc[i,"Инв"]==new_df.loc[i-1,"Инв"])& \
             (new_df.loc[i,"Дата 1"]==new_df.loc[i-1,"Дата 1"])
            ):
           # Moto4_pokaz.loc[i,"счетчик"]=Moto4_pokaz.loc[i-1,"счетчик"]+1
            new_df.loc[i,"Уменьшающийся объем"]=new_df.loc[i-1,"Уменьшающийся объем"]-min(new_df.loc[i,"Производительность"],new_df.loc[i,"Уменьшающийся объем"])
           
        
    else:
          
            new_df['Уменьшающийся объем'] = new_df['Объем']
          
    

In [31]:
i=0
#=ЕСЛИ(Уменьшающийся объем<0;P586;ЕСЛИ(Уменьшающийся объем=0;P586;ЕСЛИ(И(D587=D586;I587=I586;C587=C586;L587=L586);МИН(Уменьшающийся объем586-Уменьшающийся объем587;Уменьшающийся объем587);МИН(Уменьшающийся объем587;J587))))
for i, row in new_df.iterrows():
    if i >= 1:
        if new_df.loc[i, "Уменьшающийся объем"] <= 0:
            new_df.loc[i, "Выполненный объем"] = new_df.loc[i-1, "Выполненный объем"]
        else:
            if (new_df.loc[i, "Передел"] == new_df.loc[i-1, "Передел"]) and \
               (new_df.loc[i, "Марка техники"] == new_df.loc[i-1, "Марка техники"]) and \
               (new_df.loc[i, "Блок"] == new_df.loc[i-1, "Блок"]) and \
               (new_df.loc[i, "Инв"] == new_df.loc[i-1, "Инв"]) and \
               (new_df.loc[i, "Дата 1"] == new_df.loc[i-1, "Дата 1"]):
                    new_df.loc[i, "Выполненный объем"] = min((new_df.loc[i-1, "Уменьшающийся объем"]-new_df.loc[i, "Уменьшающийся объем"]), new_df.loc[i, "Уменьшающийся объем"])
            else:
                new_df.loc[i, "Выполненный объем"] = min(new_df.loc[i, "Уменьшающийся объем"], new_df.loc[i, 'Производительность'])
    if i == 0:
         new_df.loc[i, "Выполненный объем"]=min(new_df.loc[i, "Уменьшающийся объем"], new_df.loc[i, 'Производительность'])


In [32]:
new_df['Название месяца'] = new_df['Дата'].dt.strftime('%B').str.capitalize()

In [33]:
new_df['Месяц']=new_df['Дата'].dt.month

In [35]:
new_df['Название месяца'].unique()

array(['June', 'July', 'May', 'August', 'September'], dtype=object)

In [36]:
new_df['Название месяца'].replace({'June': 'Июнь', 'July': 'Июль', 'May': 'Май', 'August': 'Август', 'September': 'Сентябрь'})

0           Июнь
1           Июнь
2           Июнь
3           Июнь
4           Июнь
          ...   
1149    Сентябрь
1150    Сентябрь
1151    Сентябрь
1152    Сентябрь
1153    Сентябрь
Name: Название месяца, Length: 1154, dtype: object

In [37]:
pivot_table = pd.pivot_table(new_df,values=[ 'Производительность'], 
                             index=['Дата'], 
                             columns=['Марка техники+Инв'], 
                             aggfunc={ 'Производительность': 'sum'})


pivot_table = pivot_table.round(2)
pivot_table = pivot_table.fillna(0)

# установка формата вывода
pd.options.display.float_format = '{:,.0f}'.format

# применение формата для всех чисел в таблице
pivot_table = pivot_table.applymap(lambda x: '{:,.0f}'.format(x) if isinstance(x, (int, float)) else x)

# print the updated pivot table
pivot_table


Производительность                                   \
Марка техники+Инв          D-375/677 D-375/688 DM-45/967 PC-800 8/798   
Дата                                                                    
2023-05-31                     3,400         0         0            0   
2023-06-01                     2,500     2,500         0        3,534   
2023-06-02                     2,500     2,500         0        3,534   
2023-06-03                     2,500     2,500       500        3,534   
2023-06-04                     2,500     2,500       500        5,302   
...                              ...       ...       ...          ...   
2023-09-19                         0         0         0        2,400   
2023-09-20                         0         0         0        2,400   
2023-09-21                         0         0         0        2,400   
2023-09-22                         0         0         0        2,400   
2023-09-23                         0         0         0        2,400   

                                                                           \
Марка техники+Инв PC-800 8/913 БелАЗ/1082 БелАЗ/1150 БелАЗ/1155 БелАЗ/296   
Дата                                                                        
2023-05-31               3,534      1,178      1,178      1,178         0   
2023-06-01               3,448      1,149      1,149      1,149     1,178   
2023-06-02               3,491      1,164      1,164      1,164     1,178   
2023-06-03               3,491      1,164      1,164      1,164     1,178   
2023-06-04               3,491      1,164      1,164      1,164     1,767   
...                        ...        ...        ...        ...       ...   
2023-09-19                   0          0          0          0         0   
2023-09-20                   0          0          0          0         0   
2023-09-21                   0          0          0          0         0   
2023-09-22                   0          0          0          0         0   
2023-09-23                   0          0          0          0         0   

                                                     
Марка техники+Инв БелАЗ/659 БелАЗ/660 ГИТ-62 МЕ/668  
Дата                                                 
2023-05-31                0         0             0  
2023-06-01            1,178     1,178             0  
2023-06-02            1,178     1,178             0  
2023-06-03            1,178     1,178             0  
2023-06-04            1,767     1,767             0  
...                     ...       ...           ...  
2023-09-19            1,200     1,200         2,400  
2023-09-20            1,200     1,200         2,400  
2023-09-21            1,200     1,200         2,400  
2023-09-22            1,200     1,200         2,400  
2023-09-23            1,200     1,200         2,400  

[116 rows x 12 columns]

In [38]:
new_df.to_excel(os.path.join(current_dir, 'Дымный_календарь_тест.xlsx'), "Дымный")